Données d'origine : <https://data.enedis.fr/explore/dataset/prod/information/>

In [3]:
import numpy
import pandas
import iso8601

In [4]:
prod_brut = pandas.read_csv("/Users/fxjollois/Downloads/prod.csv", sep = ";")
prod_brut.head()

,Horodate,Plage de puissance injection,Filière de production,Nb points injection,Total énergie injectée (Wh),Courbe Moyenne n°1 (Wh),Indice représentativité Courbe n°1 (%),Courbe Moyenne n°2 (Wh),Indice représentativité Courbe n°2 (%),Courbe Moyenne n°1 + n°2 (Wh),Indice représentativité Courbe n°1 + n°2 (%),Jour max du mois (0/1),Semaine max du mois (0/1)
0,2020-12-31T23:30:00+01:00,P1 : ]0 - 3] kW,F2 : Bioénergies,6,NaN,NaN,S,NaN,S,NaN,S,0,0
1,2020-12-31T23:30:00+01:00,P1 : ]0 - 3] kW,F3 : Hydraulique,126,22757580.0,0.0,0,NaN,S,43.0,0,0,0
2,2020-12-31T23:30:00+01:00,P2 : ]3 - 9] kW,F4 : Eolien,96,NaN,NaN,S,NaN,S,NaN,S,0,0
3,2020-12-31T23:30:00+01:00,P3 : ]9 - 100] kW,F6 : Autres,210,534179.0,NaN,S,NaN,S,4262.0,0,0,0
4,2020-12-31T23:30:00+01:00,P5 : ]250 - 1000] kW,F5 : Solaire,578,NaN,NaN,S,NaN,S,NaN,S,0,0


## Code pour 2019-2020

In [5]:
prod = prod_brut.query('`Plage de puissance injection` == "P0 : Total toutes puissances"')
prod = prod.assign(Date = [iso8601.parse_date(d).strftime("%Y%m%d") for d in prod["Horodate"]])
prod = prod.filter(["Date", "Filière de production", "Total énergie injectée (Wh)"])
prod = prod.groupby(["Date", "Filière de production"])
prod = prod.aggregate(nb = ("Date", "count"), Production = ("Total énergie injectée (Wh)", numpy.sum))
prod = prod.reset_index()
prod = prod.query('nb == 48')
prod.head()

,Date,Filière de production,nb,Production
0,20180101,F0 : Total toutes filières,48,2.837167e+11
1,20180101,F1 : Thermique non renouvelable,48,3.810881e+10
2,20180101,F2 : Bioénergies,48,1.499302e+10
3,20180101,F3 : Hydraulique,48,1.531824e+10
4,20180101,F4 : Eolien,48,2.080416e+11


In [6]:
prod_final = pandas.pivot(prod, index = "Date", columns = "Filière de production", values = "Production")
prod_final = prod_final.drop(columns = "F0 : Total toutes filières")
prod_final = prod_final.reset_index()
prod_final.head()

Filière de production,Date,F1 : Thermique non renouvelable,F2 : Bioénergies,F3 : Hydraulique,F4 : Eolien,F5 : Solaire,F6 : Autres
0,20180101,3.810881e+10,1.499302e+10,1.531824e+10,2.080416e+11,7.040696e+09,137993136.0
1,20180102,3.862201e+10,1.457315e+10,1.579756e+10,1.525662e+11,5.392546e+09,136383120.0
2,20180103,3.812646e+10,1.414417e+10,1.525426e+10,2.081919e+11,5.503013e+09,230064480.0
3,20180104,3.852517e+10,1.486626e+10,1.398220e+10,1.905919e+11,5.847942e+09,327682464.0
4,20180105,3.891543e+10,1.491484e+10,1.366702e+10,1.140034e+11,7.785438e+09,328796976.0


## Code pour 2020-2021

In [7]:
prod = prod_brut.query('`Filière de production` == "F0 : Total toutes filières"')
prod = prod.assign(Date = [iso8601.parse_date(d).strftime("%Y%m%d") for d in prod["Horodate"]])
prod = prod.filter(["Date", "Plage de puissance injection", "Total énergie injectée (Wh)"])
prod = prod.groupby(["Date", "Plage de puissance injection"])
prod = prod.aggregate(nb = ("Date", "count"), Production = ("Total énergie injectée (Wh)", numpy.sum))
prod = prod.reset_index()
prod = prod.query('nb == 48')
prod.head()

,Date,Plage de puissance injection,nb,Production
0,20180101,P0 : Total toutes puissances,48,2.837167e+11
1,20180101,P1 : ]0 - 3] kW,48,2.077390e+09
2,20180101,P2 : ]3 - 9] kW,48,3.757243e+08
3,20180101,P3 : ]9 - 100] kW,48,1.550615e+09
4,20180101,P4 : ]100 - 250] kW,48,2.662994e+09


In [8]:
prod_final = pandas.pivot(prod, index = "Date", columns = "Plage de puissance injection", values = "Production")
prod_final = prod_final.drop(columns = "P0 : Total toutes puissances")
prod_final = prod_final.reset_index()
prod_final.head()

Plage de puissance injection,Date,P1 : ]0 - 3] kW,P2 : ]3 - 9] kW,P3 : ]9 - 100] kW,P4 : ]100 - 250] kW,P5 : ]250 - 1000] kW,P6 : ]1000 - 12000] kW,P7 : > 12000 kW
0,20180101,2.077390e+09,375724284.0,1.550615e+09,2.662994e+09,6.306990e+09,2.425381e+11,2.820488e+10
1,20180102,2.051658e+09,303267043.0,1.292679e+09,2.467395e+09,6.221920e+09,1.935935e+11,2.123369e+10
2,20180103,2.009292e+09,331861128.0,1.380157e+09,2.403259e+09,5.862378e+09,2.415548e+11,2.798439e+10
3,20180104,1.979063e+09,291015949.0,1.239656e+09,2.282586e+09,5.689795e+09,2.274995e+11,2.523585e+10
4,20180105,2.054262e+09,387378453.0,1.573815e+09,2.529303e+09,5.882376e+09,1.596449e+11,1.761923e+10


In [9]:
prod_final.to_csv("FilieresProduction.csv", index = False)
